<a href="https://colab.research.google.com/github/adalbertii/Modele-klasyfikacyjne/blob/main/keras_classification_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Klasyfikacja binarna**

Oparta na modelu neuronowym



Step 1 - Loading the Required Libraries and Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt


import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

**Pobranie danych i ich uproszczona analiza**

In [ ]:
# zamontować dysk Google
df = pd.read_csv('/content/drive/MyDrive/dane/diabetes.csv')
print(df.shape)

In [ ]:

df.head()

In [ ]:
# analiza statystyczna zbioru danych
df.describe()

Step 3 - Creating Arrays for the Features and the Response Variable.

In [9]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [12]:
target_column = ['Outcome']
predictors = list(set(list(df.columns))-set(target_column))
predictors

['Age',
 'DiabetesPedigreeFunction',
 'Insulin',
 'Glucose',
 'Pregnancies',
 'BMI',
 'BloodPressure',
 'SkinThickness']

In [ ]:
# uproszczone skalowanie wartości
df[predictors] = df[predictors]/df[predictors].max()
df.describe()

In [13]:
# tworzy tablice np.array
X = df[predictors].values
y = df[target_column].values

In [14]:
X.shape

(768, 8)

In [15]:
y.shape

(768, 1)

In [19]:
# podzial zbioru na dane treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print('X_train:', X_train.shape); print('X_test:', X_test.shape);print('y_train:', y_train.shape);print('Y_test:',y_test.shape)

X_train: (537, 8)
X_test: (231, 8)
y_train: (537, 1)
Y_test: (231, 1)


In [20]:
#Ponieważ nasza zmienna docelowa reprezentuje kategorię binarną, która została zakodowana jako liczby 0 i 1,
#będziemy musieli ją zakodować.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

count_classes = y_test.shape[1]
print('Liczba klas:', count_classes)

Liczba klas: 2


In [21]:
y_train[:5]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [22]:
print(y_train.shape)
y_train

(537, 2)


array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

**Definicja, kompilacja i uczenie modelu neuronowego**

In [23]:
X_train.shape

(537, 8)

In [24]:
#definicja modelu
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=8)) # objaśnić input_dim
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))# objaśnić funkcje aktywacji softmax


# Podczas pracy nad problemem uczenia maszynowego lub głębokiego uczenia,
# funkcje strat/kosztów są wykorzystywane do optymalizacji modelu podczas szkolenia.
# Celem jest prawie zawsze zminimalizowanie funkcji strat.
# Im niższa strata, tym lepszy model. Cross-Entropy loss jest najważniejszą funkcją kosztu.
# Służy ona do optymalizacji modeli klasyfikacyjnych.

In [25]:
# kompilacja modelu
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               4500      
                                                                 
 dense_1 (Dense)             (None, 100)               50100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 59752 (233.41 KB)
Trainable params: 59752 (233.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# uczenie modelu
model.fit(X_train, y_train, epochs=20, verbose=1)


Epoch 1/20
17/17 [==============================] - 1s 3ms/step - loss: 0.6555 - accuracy: 0.6667
Epoch 2/20
17/17 [==============================] - 0s 3ms/step - loss: 0.6243 - accuracy: 0.6667
Epoch 3/20
17/17 [==============================] - 0s 3ms/step - loss: 0.5974 - accuracy: 0.6983
Epoch 4/20
17/17 [==============================] - 0s 3ms/step - loss: 0.5736 - accuracy: 0.7020
Epoch 5/20
17/17 [==============================] - 0s 4ms/step - loss: 0.5500 - accuracy: 0.7318
Epoch 6/20
17/17 [==============================] - 0s 3ms/step - loss: 0.5339 - accuracy: 0.7486
Epoch 7/20
17/17 [==============================] - 0s 3ms/step - loss: 0.5104 - accuracy: 0.7505
Epoch 8/20
17/17 [==============================] - 0s 3ms/step - loss: 0.4905 - accuracy: 0.7635
Epoch 9/20
17/17 [==============================] - 0s 3ms/step - loss: 0.4722 - accuracy: 0.7709
Epoch 10/20
17/17 [==============================] - 0s 3ms/step - loss: 0.4826 - accuracy: 0.7765
Epoch 11/20
17/17 [

**Ewaluacja modelu**

In [27]:
# pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))

pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

Accuracy on training data: 0.7988826632499695% 
 Error on training data: 0.20111733675003052
8/8 [==============================] - 0s 3ms/step
Accuracy on test data: 0.7705627679824829% 
 Error on test data: 0.2294372320175171
